In [1]:
import glob
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import nrrd
import os

%matplotlib inline


In [ ]:
np.seterr(divide=None)
np.divide(1.0, 0)

In [ ]:
add(**a)

In [ ]:
def nrrd_reader(file_path):
    _nrrd = nrrd.read(file_path)
    data = _nrrd[0]
    header = _nrrd[1]
    return data, header

In [ ]:

def merge_annotations(annotations_root='/home/abhijit/Jyotirmay/thesis/Manual annotations/Finals/UKB/*', classes=['spleen', 'liver']):
    paths = glob.glob(annotations_root)

    for p in paths:
        id_ = p.split('/')[-1]
        print(id_)
        annotations = glob.glob(p+'/**')
        if len(annotations) > len(classes):
            print('skipped')
            continue

        merged_annotations = None
        data_ = {c:None for c in classes}
        
        for a in annotations:
            data, header = nrrd_reader(a)
            
#             _ = [data_[c] = np.multiply(i, data) for i, c in enumerate(classes) if c.upper() in a.upper()]
            
            if 'SPLEEN' in a.upper():
                data_['spleen'] = data
            elif 'LIVER' in a.upper():
                data_['liver'] = np.multiply(2, data)
                
        if data_['spleen'] is None or data_['liver'] is None:
            print('skipped')
            continue

        merged_annotations = np.add(data_['spleen'],data_['liver'])
        img = nb.Nifti1Image(merged_annotations, np.eye(4))
        filename = '/'.join(annotations[0].split('/')[:-2])+'/'+id_+'_merged_annotations.nii.gz'
        nb.save(img, filename)
        
merge_annotations()


In [ ]:
x = '{0} helps {0}{1}'.format('JJ', 'c')
print(x)

In [ ]:
def decorator_with_arguments(function):
    def wrapper_accepting_arguments(arg1, arg2):
        print("My arguments are: {0}, {1}".format(arg1,arg2))
        function(arg1, arg2)
    return wrapper_accepting_arguments


@decorator_with_arguments
def cities(city_one):
    print("Cities I love are {0} and {1}".format(city_one, city_two))

cities("Nairobi", "Accra")

In [12]:

def nrrd_reader(file_path):
    _nrrd = nrrd.read(file_path)
    data = _nrrd[0]
    header = _nrrd[1]
    return data, header

def nibabel_reader(file_path):
    volume_nifty = nb.load(file_path)
    volume = volume_nifty.get_fdata()
    return volume, volume_nifty.header

def merge_annotations(id_):
    print(id_)
    p = f'/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/{id_}'
    labels = ["Background", "Spleen", "Liver"]
    classes = labels[1:]  # Excluding background class.
    excluded_volumes = []
    label_dir = p
    processed_extn = '.nii'
    header = None
    try:
        annotations = glob.glob(p + '/**')
        if len(annotations) > len(classes)+1:
            print('skipped 1', len(annotations) )
            excluded_volumes.append(id_)
            return False

        data_ = {c.lower(): None for c in classes}
        print(annotations)
        for a in annotations:
            print(a)
            if 'COMP_MASK' in a.upper():
                print('skipped_comp_mask')
                continue
            ext = a.split('.')[-1]
            if ext == "nrrd":
                data, header = nrrd_reader(a)
                if len(data.shape) > 3:
                    data = data[:,:,:,0]
            elif ext in ['gz', 'mgz', 'nii']:
                data, header = nibabel_reader(a)
                if len(data.shape) > 3:
                    data = data[:,:,:,0]

            if 'SPLEEN2D' in a.upper():
                data_['spleen'] = data
            elif 'LIVER2D' in a.upper():
                data_['liver'] = np.multiply(2, data)

        if data_['spleen'] is None or data_['liver'] is None:
            excluded_volumes.append(id_)
            print('skipped 2')
            return False
        print(data_['spleen'].shape, data_['liver'].shape)
        print(np.unique(data_['spleen']), np.unique(data_['liver']))

        merged_annotations = np.add(data_['spleen'], data_['liver'])
        affine = affine = header.get_best_affine()
        print(affine, header)
        img = nb.Nifti1Image(merged_annotations, affine, header)
        filename = os.path.join(label_dir, 'comp_mask' + processed_extn)
        print(filename)
        nb.save(img, filename)
        
        cdata = nb.load(filename)
        header = cdata.header
        data = cdata.get_fdata()
        
        print('check if all int:', np.unique(data), np.unique(np.round(data)))
    except Exception as e:
        print(e)
        excluded_volumes.append(id_)
    print(excluded_volumes)

In [13]:
cf = ['KORA2460543', 'KORA2456289', 'KORA2452381', 'KORA2459499', 'KORA2455951', 'KORA2460382', 'KORA2459095',
      'KORA2460216', 'KORA2453108', 'KORA2458057', 'KORA2452542', 'KORA2452274', 'KORA2458926', 'KORA2460796', 
      'KORA2460611', 'KORA2456379', 'KORA2459818', 'KORA2456928', 'KORA2453290', 'KORA2455935', 'KORA2459258',
      'KORA2458342', 'KORA2456199', 'KORA2453470', 'KORA2456202', 'KORA2456661', 'KORA2453218', 'KORA2460360', 
      'KORA2452885', 'KORA2459780', 'KORA2453172', 'KORA2453374', 'KORA2459994', 'KORA2452801', 'KORA2452834',
      'KORA2460238']
cf = ['KORA2453108',  'KORA2452542',  'KORA2453374']
cf = ['KORA2453108']
for c in cf:
    merge_annotations(c)
#     break

KORA2453108
['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453108/liver2d.nii', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453108/spleen2d.nii', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453108/comp_mask.nii']
/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453108/liver2d.nii
/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453108/spleen2d.nii
/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453108/comp_mask.nii
skipped_comp_mask
(293, 426, 160) (293, 426, 160)
[0. 1.] [0. 2.]
[[ -1.66666663   0.           0.         254.80715942]
 [  0.           0.          -1.69999695 111.92337799]
 [  0.          -1.66666663   0.         -21.8153553 ]
 [  0.           0.           0.           1.        ]] <class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      

In [ ]:
round(0.9996)

In [ ]:
round(2.)